# Building a neutral net for coolaborative filtering

In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf

In [2]:
#Data 
# # colname = ['user_id','item_id','rating','Time','Location','Companion']
# df = pd.read_csv('Movie_DePaulMovie/ratings.txt', sep=',')
import graphlab as gl
sf = gl.SFrame.read_csv('Movie_DePaulMovie/ratings.txt', sep=',')

This non-commercial license of GraphLab Create for academic use is assigned to hramachandran@hawk.iit.edu and will expire on June 26, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1500747755.log


Finished parsing file /home/harish/PycharmProjects/Recommender_System/movieLens/RecSys_DeepLearning/Movie_DePaulMovie/ratings.txt

Parsing completed. Parsed 100 lines in 0.020426 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,int,str,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /home/harish/PycharmProjects/Recommender_System/movieLens/RecSys_DeepLearning/Movie_DePaulMovie/ratings.txt

Parsing completed. Parsed 5043 lines in 0.014755 secs.

In [3]:
sf.tail()

userid,itemid,rating,Time,Location,Companion
1082,tt1637706,2,Weekday,Home,Partner
1082,tt0343660,2,Weekday,Home,Partner
1082,tt1133985,1,Weekday,Home,Partner
1082,tt1099212,1,Weekday,Home,Partner
1082,tt3793764,1,Weekend,Home,Partner
1082,tt0413267,1,Weekend,Home,Partner
1082,tt1637706,2,Weekend,Home,Partner
1082,tt0343660,1,Weekend,Home,Partner
1082,tt1133985,1,Weekend,Home,Partner
1082,tt1099212,1,Weekend,Home,Partner


In [4]:
sf.column_types()#note object(in case of pandas df) data types are not convertible when using SFrame datastructure[https://forum.turi.com/discussion/1925/]

[int, str, int, str, str, str]

In [5]:
# Total no.of users and movies
users = sf['userid'].unique().shape[0]
items = sf['itemid'].unique().shape[0]
print "total no. of users, movies:"+ str(users)+","+str(items)

total no. of users, movies:97,79


In [6]:
#train test split:(80/20 split)
SF_train, SF_test = sf.random_split(.8, seed=5)
print(len(SF_train), len(SF_test))


(4030, 1013)


In [7]:
#Modeling: Collaborative Filtering

In [8]:
## Item similarity model

In [9]:
itemSimilarity_model = gl.item_similarity_recommender.create(SF_train, user_id='userid', item_id='itemid', target='rating', similarity_type='pearson')

Recsys training: model = item_similarity

Warning: Ignoring columns Time, Location, Companion;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 4030 observations with 97 users and 79 items.

...

+--------------------------------+------------+

Setting up lookup tables.

Processing data in one pass using dense lookup tables.

+-------------------------------------+------------------+-----------------+

| Elapsed Time (Constructing Lookups) | Total % Complete | Items Processed |

In [10]:
nn = itemSimilarity_model.get_similar_items(k=1)
nn.print_rows(num_rows=25)

+-------------------------------------+------------------+-----------------+

| 2.299ms                             | 0                | 0               |

| 8.999ms                             | 100              | 79              |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.0124s

+-----------+-----------+----------------+------+
|   itemid  |  similar  |     score      | rank |
+-----------+-----------+----------------+------+
| tt1499658 | tt1478338 | 0.29923593998  |  1   |
| tt0405422 | tt2251217 | 0.243927121162 |  1   |
| tt0109830 | tt0110357 | 0.420491039753 |  1   |
| tt0088763 | tt0109830 | 0.41255402565  |  1   |
| tt0133093 | tt1657301 | 0.33416980505  |  1   |
| tt0800369 | tt0319262 | 0.344946742058 |  1   |
| tt0266543 | tt0110357 | 0.253341197968 |  1   |
| tt1632708 | tt1596343 | 0.320347070694 |  1   |
| tt4052394 | tt1133985 | 0.271005630493 |  1   |
| tt1596343 | tt1632708 | 0.320347070694 |  1   |
| tt2574698 | tt0266543 | 0.234904289246 |  1   |
| tt0454848 | tt1232829 | 0.26954716444  |  1   |
| tt0378194 | tt0114148 | 0.349371314049 |  1   |
| tt0211915 | tt1453405 | 0.198636829853 |  1   |
| tt0169547 | tt0327084 | 0.311162769794 |  1   |
| tt0454876 | tt0110357 | 0.259337067604 |  1   |
| tt0181689 | tt0213149 | 0.31661683321  |  1   |


In [11]:
itemSimilarity_recomm = itemSimilarity_model.recommend(k=1)
itemSimilarity_recomm.print_rows(num_rows=25)

+--------+-----------+---------------+------+
| userid |   itemid  |     score     | rank |
+--------+-----------+---------------+------+
|  1123  | tt0315733 | 4.36417872508 |  1   |
|  1090  | tt0315733 | 4.33333333333 |  1   |
|  1088  | tt0315733 | 4.33333333333 |  1   |
|  1110  | tt0315733 | 4.32672772805 |  1   |
|  1017  | tt1375666 | 4.44378136334 |  1   |
|  1021  | tt0315733 | 4.30754718894 |  1   |
|  1095  | tt0315733 | 4.33333333333 |  1   |
|  1106  | tt0315733 | 4.33333333333 |  1   |
|  1102  | tt0315733 | 4.33333333333 |  1   |
|  1117  | tt0315733 | 4.39074804386 |  1   |
|  1091  | tt0315733 |  4.3335228314 |  1   |
|  1025  | tt0315733 | 4.33333333333 |  1   |
|  1108  | tt0109830 | 4.39838058395 |  1   |
|  1020  | tt0315733 | 4.32684457799 |  1   |
|  1003  | tt0315733 |  4.3166179217 |  1   |
|  1019  | tt0315733 | 4.34729182919 |  1   |
|  1046  | tt0315733 |  4.3212606286 |  1   |
|  1067  | tt0315733 | 4.31862295171 |  1   |
|  1018  | tt0315733 | 4.304022130

In [12]:
#lets check dataset for itemid = tt0109830 alone:
userid_ofValue1 = sf[sf['itemid'] == 'tt0109830']

In [13]:
userid_ofValue1.sort(sort_columns='rating', ascending=False).print_rows(num_rows=60)

+--------+-----------+--------+---------+----------+-----------+
| userid |   itemid  | rating |   Time  | Location | Companion |
+--------+-----------+--------+---------+----------+-----------+
|  1123  | tt0109830 |   5    |   None  |   None   |    None   |
|  1025  | tt0109830 |   5    |   None  |   None   |    None   |
|  1082  | tt0109830 |   5    |   None  |   None   |    None   |
|  1058  | tt0109830 |   5    |   None  |   None   |    None   |
|  1116  | tt0109830 |   5    | Weekend |  Cinema  |  Partner  |
|  1116  | tt0109830 |   5    | Weekday |  Cinema  |   Alone   |
|  1116  | tt0109830 |   5    |   None  |   None   |    None   |
|  1059  | tt0109830 |   5    | Weekend |  Cinema  |   Family  |
|  1098  | tt0109830 |   5    |   None  |   None   |    None   |
|  1002  | tt0109830 |   5    |   None  |   None   |    None   |
|  1069  | tt0109830 |   5    | Weekend |  Cinema  |  Partner  |
|  1002  | tt0109830 |   5    | Weekend |   Home   |  Partner  |
|  1113  | tt0109830 |   

## Prediction/recommendation for test case:
[https://turi.com/products/create/docs/generated/graphlab.recommender.item_similarity_recommender.create.html#graphlab.recommender.item_similarity_recommender.create]

In [14]:
itemSimilarity_model.predict(SF_test)
itemSimilarity_model.recommend(k=2).print_rows(num_rows=50, num_columns=4)

+--------+-----------+---------------+------+
| userid |   itemid  |     score     | rank |
+--------+-----------+---------------+------+
|  1123  | tt0315733 | 4.36417872508 |  1   |
|  1123  | tt1375666 | 4.22757187224 |  2   |
|  1090  | tt0315733 | 4.33333333333 |  1   |
|  1090  | tt1375666 | 4.22517167835 |  2   |
|  1088  | tt0315733 | 4.33333333333 |  1   |
|  1088  | tt0109830 |  4.2661312336 |  2   |
|  1110  | tt0315733 | 4.32672772805 |  1   |
|  1110  | tt1375666 | 4.18607777096 |  2   |
|  1017  | tt1375666 | 4.44378136334 |  1   |
|  1017  | tt0109830 | 4.33797483785 |  2   |
|  1021  | tt0315733 | 4.30754718894 |  1   |
|  1021  | tt1375666 | 4.23435307565 |  2   |
|  1095  | tt0315733 | 4.33333333333 |  1   |
|  1095  | tt1375666 | 4.27344216145 |  2   |
|  1106  | tt0315733 | 4.33333333333 |  1   |
|  1106  | tt1375666 |  4.2651803527 |  2   |
|  1102  | tt0315733 | 4.33333333333 |  1   |
|  1102  | tt1375666 | 4.26863509155 |  2   |
|  1117  | tt0315733 | 4.390748043

In [15]:
#Checking the performance of our recommender ; here using 4,5 as the best recommendation
SF_test[SF_test['rating'] > 3].print_rows(num_rows=50)

+--------+-----------+--------+---------+----------+-----------+
| userid |   itemid  | rating |   Time  | Location | Companion |
+--------+-----------+--------+---------+----------+-----------+
|  1088  | tt0405422 |   4    |   None  |   None   |    None   |
|  1088  | tt0133093 |   5    |   None  |   None   |    None   |
|  1017  | tt1190080 |   5    |   None  |   None   |    None   |
|  1017  | tt0454876 |   5    |   None  |   None   |    None   |
|  1021  | tt1099212 |   5    |   None  |   None   |    None   |
|  1095  | tt1478338 |   5    |   None  |   None   |    None   |
|  1106  | tt1657301 |   4    |   None  |   None   |    None   |
|  1106  | tt0407304 |   4    |   None  |   None   |    None   |
|  1117  | tt0120338 |   5    |   None  |   None   |    None   |
|  1117  | tt3510098 |   4    |   None  |   None   |    None   |
|  1117  | tt1375666 |   5    |   None  |   None   |    None   |
|  1091  | tt0111161 |   5    |   None  |   None   |    None   |
|  1025  | tt0413267 |   